**В данном файле мною реализованы classification/regression metrics & процедура алгоритма DecisionTree. Таким образом, этот файл включает не "from sklearn.... import ....", а как работает тот или иной алгоритм "под капотом".**
* метрики классификации: AP, Precision, Recall, F_beta, Accuracy, Cross-entropy 
* метрики регрессии: MSE, MAE, MSLE, Quantile standart error
* Использование алгоритма DecisionTree для обучения и предикта

> В дополнении хочется отметить, что алгоритм дерева решений был написан в .py, чтобы не загромождать текущий файл. Его можно переписать под конкретную задачу или добавить/убрать гиперпараметры, которые реализованы там.

In [ ]:
# Импортирование необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt

In [1]:
# Импортирование класса из файла DecisionTreeClass, чтобы решение выглядело более компактно
# нужно, чтобы файл .py был в одной директории с юпитер ноутбуком
from DecisionTreeClass import DecisionTree

In [ ]:
# Реализация метрик классификации
def average_precision(y_true, y_pred):
    y_true_sorted, y_pred_sorted = zip(*sorted(zip(y_true, y_pred), key=lambda x: x[1], reverse=True))
    cumsum = np.cumsum(y_true_sorted)
    precision = cumsum / (np.arange(len(y_true_sorted)) + 1)
    recall = cumsum / np.sum(y_true)
    ap = np.sum(precision[:-1] * (recall[1:] - recall[:-1]))
    return ap

def precision(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    false_positives = np.sum((y_true == 0) & (y_pred == 1))
    
    denominator = true_positives + false_positives
    if denominator == 0:
        return 0.0  
    else:
        return true_positives / denominator

def recall(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    false_negatives = np.sum((y_true == 1) & (y_pred == 0))
    
    denominator = true_positives + false_negatives
    if denominator == 0:
        return 0.0  
    else:
        return true_positives / denominator

def f_beta(y_true, y_pred, beta=1):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    
    denominator = (beta**2 * precision_value) + recall_value
    if denominator == 0:
        return 0.0  
    else:
        return (1 + beta**2) * (precision_value * recall_value) / denominator

def accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)
    total_samples = len(y_true)
    return correct_predictions / total_samples

def cross_entropy(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    cross_entropy_value = - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return cross_entropy_value

In [ ]:
# Реализация метрик регрессии
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mean_squared_log_error(y_true, y_pred):
    epsilon = 1e-10
    if any(y <= 0 for y in y_true) or any(y <= 0 for y in y_pred):
        return 0
    
    return np.mean((np.log(y_true + epsilon) - np.log(y_pred + epsilon))**2)

def quantile_standard_error(y_true, y_pred, q=0.5):
    quantile_error = np.abs(y_true - y_pred) - q * (y_true - y_pred)
    return np.mean(np.maximum(quantile_error, 0))

In [ ]:
# Функции для создания датасетов с заданными уже "начальными" параметрами, которые можно изменять
def generate_classification(num_datasets=10, samples=1000, features=20, random_state=None):
    datasets = []
    for _ in range(num_datasets):
        X, y = make_classification(n_samples=samples, n_features=features, random_state=random_state)
        datasets.append((X, y))
    return datasets

def generate_regression(num_datasets=10, samples=1000, features=20, random_state=None):
    datasets = []
    for _ in range(num_datasets):
        X, y = make_regression(n_samples=samples, n_features=features, random_state=random_state)
        datasets.append((X, y))
    return datasets

# Функция для разделения выборки ( train, test, val)
test_size = ...
validation_size = ...

def split_dataset(X, y, test_size = test_size, validation_size = validation_size, random_state=None):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(test_size + validation_size), random_state=random_state)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=validation_size/(test_size + validation_size), random_state=random_state)
    return X_train, X_test, X_val, y_train, y_test, y_val


# Обучение и оценка моделей classification
def train_and_evaluate_classifier(X_train, X_val, y_train, y_val, criterion, splitter, leaf,mx_features):
    model = DecisionTree(criterion = criterion , splitter = splitter , min_samples_leaf = leaf, max_features = mx_features, task = 'classification')
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    
    ap = average_precision(y_val, y_val_pred)
    accuracy_score = accuracy(y_val, y_val_pred)
    precision_score = precision(y_val, y_val_pred)
    recall_score = recall(y_val, y_val_pred)
    fbeta_score = f_beta(y_val, y_val_pred, beta=1)
    cross_entropy_score = cross_entropy(y_val, y_val_pred)
    return ap, accuracy_score, precision_score, recall_score, fbeta_score, cross_entropy_score

# Обучение и оценка моделей regression
def train_and_evaluate_regressor(X_train, X_val, y_train, y_val, criterion, splitter, leaf,mx_features):
    model = DecisionTree(criterion = criterion , splitter = splitter , min_samples_leaf = leaf, max_features = mx_features,task = 'regression')
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_val_pred)
    mae = mean_absolute_error(y_val, y_val_pred)
    msle = mean_squared_log_error(y_val, y_val_pred)
    qse = quantile_standard_error(y_val, y_val_pred)
    return mse, mae, msle, qse

In [ ]:
# Создаем 'n' датасетов для обучения и предикта нашего "одинокого дерева"

# args для генерации(можно записать в dict и передать в функцию через **args или через знак "=" )
num_datasets = ...
samples = ...
features = ...
random_state = ... #optional


classification_datasets = generate_classification(num_datasets = num_datasets) # Пример 
regression_datasets = generate_regression(num_datasets = num_datasets) # Пример 

# При обучении и подсчете метрик реализован обычный цикл для наглядности работы алгоритма
# С данными функциями можно реализовать более сложные конструкции, которые будут требоваться в вашем случае

# Классификация(можно изменить значения гиперпарам)
cl_metrics_criterion = ["gini", "entropy"]
cl_metrics_splitter = ['best', 'random']
cl_metrics_mx_features = [None,'auto','log2','sqrt']
min_samples_leaf = [2,3,4]
cl_results = []

for dataset in classification_datasets:
    X, y = dataset
    X_train, X_test, X_val, y_train, y_test, y_val = split_dataset(X, y, random_state=42)

    for criterion, splitter,mx_features ,leaf in zip(cl_metrics_criterion,cl_metrics_splitter,cl_metrics_mx_features,min_samples_leaf):
        ap, accuracy_score, precision_score, recall_score, fbeta_score, cross_entropy_score = train_and_evaluate_classifier(X_train, X_val, y_train, y_val, criterion, splitter, leaf, mx_features)
        cl_results.append((criterion, splitter, mx_features ,leaf , ap, accuracy_score, precision_score, recall_score, fbeta_score, cross_entropy_score))

# Регрессия(можно изменить значения гиперпарам)
reg_metrics_criterion = ["mse"]
reg_metrics_splitter = ['best', 'random']
reg_metrics_mx_features = [None,'auto','log2','sqrt']
reg_results = []

for dataset in regression_datasets:
    X, y = dataset
    X_train, X_test, X_val, y_train, y_test, y_val = split_dataset(X, y, random_state=42)

    for criterion, splitter,mx_features ,leaf in zip(reg_metrics_criterion,reg_metrics_splitter,reg_metrics_mx_features,min_samples_leaf):
        mse, mae, msle, qse = train_and_evaluate_regressor(X_train, X_val, y_train, y_val, criterion, splitter, leaf, mx_features)
        reg_results.append((criterion, splitter, mx_features ,leaf, mse, mae, msle, qse))

In [ ]:
# Выводим значения метрик после обучения
classification_metrics_indices = [4, 5, 6, 7, 8]
regression_metrics_indices = [4, 5, 6, 7]

# Функция для агрегации метрик
def aggregate_metrics(metrics):
    return np.mean(metrics)

sorted_classification_results = sorted(cl_results, key=lambda x: aggregate_metrics([x[i] for i in classification_metrics_indices]), reverse=True)
sorted_regression_results = sorted(reg_results, key=lambda x: aggregate_metrics([x[i] for i in regression_metrics_indices]))


print("Результаты классификации:")
for result in sorted_classification_results:
    print(result)

print("\nРезультаты регрессии:")
for result in sorted_regression_results:
    print(result)

In [ ]:
# Выбор гиперпараметров с лучшим скором
best_classification_criterion = sorted_classification_results[0]
print("Лучшие гиперпараметры модели для классификации:", best_classification_criterion)

# Выбор лучшего критерия для регрессии
best_regression_criterion = sorted_regression_results[-1]
print("Лучшие гиперпараметры модели для регрессии:", best_regression_criterion)
